# K-Means Clustering (Maharashtra Dataset)

## Importing Libraries

In [326]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import geocoder
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import sklearn
from sklearn.cluster import KMeans

## Scraping Data from Wikipedia

In [327]:
html = urlopen("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Mumbai")
bsobj = BeautifulSoup(html,'lxml')

In [328]:
# appending names in a list
nb_list = []
for row in bsobj.find("div",{"class":"mw-category"}).findAll("li"):
    nb_list.append(row.text)
nb_list[1:5]    

['Aarey Milk Colony', 'Agripada', 'Altamount Road', 'Amboli, Mumbai']

In [329]:
# creating a dataframe with nb names
df_nb = pd.DataFrame(nb_list[1:],columns=['Neighborhood'])
df_nb.head()

,Neighborhood
0,Aarey Milk Colony
1,Agripada
2,Altamount Road
3,"Amboli, Mumbai"
4,Amrut Nagar


In [330]:
# seeing num of rows and cols
df_nb.shape 

(134, 1)

In [331]:
# defining function to get latitude and longitude values
def get_latlng(neighborhood):
    latlng = None
    while(latlng is None):
        g = geocoder.arcgis('{}, Maharashtra, India'.format(neighborhood))
        latlng = g.latlng
        return latlng        

In [332]:
latlng = [get_latlng(nb) for nb in df_nb['Neighborhood'].tolist()]

## Cleaning and Formatting

In [333]:
df_ll = pd.DataFrame(latlng, columns=['Lat','Long'])

#assigning lat and long values in dataset

df_nb['Latitude'] = df_ll['Lat']
df_nb['Longitude'] = df_ll['Long']
df_nb.head()

,Neighborhood,Latitude,Longitude
0,Aarey Milk Colony,19.170268,72.871072
1,Agripada,18.976280,72.826150
2,Altamount Road,18.964338,72.807838
3,"Amboli, Mumbai",19.129060,72.846440
4,Amrut Nagar,18.993800,73.992600


In [334]:
# checking null
df_nb.isna().sum()

Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

## Using Foursquare API to check venues in these neighborhoods

In [335]:
client_id = 'ITKVISZUN50K5OIU5THJ0A330ABDVLFMOSX4NUB41ESFVYSA'
client_secret = 'MACPFVUEFU3BEHD4YZREQUBDEBFX0JFOPQ0YLIEBRHWQ0ELC'
ver = '20200701'
lim = 100

In [336]:
import requests
radius = 1000

venues = []

for lat, long, neighborhood in zip(df_nb['Latitude'], df_nb['Longitude'], df_nb['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(client_id,client_secret,ver,lat,long,radius,lim)

venues = []

for lat, long, neighborhood in zip(df_nb['Latitude'], df_nb['Longitude'], df_nb['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(client_id,client_secret,ver,lat,long,radius,lim)
    
    # make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [337]:
df_venue = pd.DataFrame(venues)
df_venue.columns=['Neighborhood','Latitude','Longitude','VenueName','VenueLat','VenueLng','VenueCategory']
print(df_venue.shape)
df_venue.head(40)

(9370, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLat,VenueLng,VenueCategory
0,Aarey Milk Colony,19.170268,72.871072,Green Court Restaurant and Bar,19.175463,72.868763,Bar
1,Aarey Milk Colony,19.170268,72.871072,Cafe Coffee Day,19.172701,72.874219,Café
2,Aarey Milk Colony,19.170268,72.871072,Java Grind,19.172765,72.865065,Coffee Shop
3,Aarey Milk Colony,19.170268,72.871072,Domino's Pizza,19.172000,72.874000,Pizza Place
4,Aarey Milk Colony,19.170268,72.871072,McDonald's,19.173616,72.863839,Fast Food Restaurant
5,Aarey Milk Colony,19.170268,72.871072,Natural Ice Cream,19.174904,72.867982,Ice Cream Shop
6,Aarey Milk Colony,19.170268,72.871072,Ratnagiri Malwani Food,19.175969,72.874268,Seafood Restaurant
7,Aarey Milk Colony,19.170268,72.871072,Punjab Lion,19.172653,72.874602,Indian Restaurant
8,Aarey Milk Colony,19.170268,72.871072,Classic Restaurant,19.175636,72.869390,Restaurant
9,Aarey Milk Colony,19.170268,72.871072,kerkars Gym,19.174859,72.867884,Gym


## Using one-hot encoding 

In [338]:
# making values float
df_oh = pd.get_dummies(df_venue[['VenueCategory']], prefix="", prefix_sep="")
for col in df_oh.columns.values.tolist():
    print(col)
    df_oh[col] = df_oh[col].astype('float')
    print(df_oh[col].dtype)
    print('---------------------------')

ATM
float64
---------------------------
Airport
float64
---------------------------
Airport Service
float64
---------------------------
American Restaurant
float64
---------------------------
Amphitheater
float64
---------------------------
Antique Shop
float64
---------------------------
Aquarium
float64
---------------------------
Arcade
float64
---------------------------
Art Gallery
float64
---------------------------
Arts & Crafts Store
float64
---------------------------
Arts & Entertainment
float64
---------------------------
Asian Restaurant
float64
---------------------------
Athletics & Sports
float64
---------------------------
Australian Restaurant
float64
---------------------------
BBQ Joint
float64
---------------------------
Bagel Shop
float64
---------------------------
Bakery
float64
---------------------------
Bank
float64
---------------------------
Bar
float64
---------------------------
Basketball Court
float64
---------------------------
Beach
float64
-----------

float64
---------------------------
South Indian Restaurant
float64
---------------------------
Southern / Soul Food Restaurant
float64
---------------------------
Spa
float64
---------------------------
Spanish Restaurant
float64
---------------------------
Sporting Goods Shop
float64
---------------------------
Sports Bar
float64
---------------------------
Sports Club
float64
---------------------------
Stadium
float64
---------------------------
Steakhouse
float64
---------------------------
Supermarket
float64
---------------------------
Sushi Restaurant
float64
---------------------------
Tea Room
float64
---------------------------
Tennis Court
float64
---------------------------
Tex-Mex Restaurant
float64
---------------------------
Thai Restaurant
float64
---------------------------
Theater
float64
---------------------------
Theme Park
float64
---------------------------
Toy / Game Store
float64
---------------------------
Track
float64
---------------------------
Trail
float

In [339]:
df_oh['Neighborhood'] = df_venue['Neighborhood']
# making nb first col
fix_cols = [df_oh.columns[-1]] + list(df_oh.columns[:-1])
df_oh = df_oh[fix_cols]
print(df_oh.shape)
df_oh.head(10)

(9370, 231)


,Zoo,ATM,Airport,Airport Service,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Cafeteria,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,...,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Luggage Store,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train,Train Station,Travel Lounge,Tree,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Aarey Milk Colony,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [340]:
#grouping weight index of each neighborhood with mean
df_oh = df_oh.groupby('Neighborhood').mean().reset_index()
df_oh.head(20)

,Neighborhood,Zoo,ATM,Airport,Airport Service,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Cafeteria,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,...,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Luggage Store,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Nature Preserve,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train,Train Station,Travel Lounge,Tree,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Aarey Milk Colony,0.000000,0.00,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.066667,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.066667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.066667,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.066667,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0666

In [341]:
# cleaning
df_oh.dropna(inplace=True)
df_oh.shape

(125, 231)

## Exploring the amenities in diff places

In [342]:
# Checking Categories that most places have in common 
for col in df_oh.columns.values.tolist():
    print(col)
    print('----------------------------')
    print(len(df_oh[df_oh[col] != 0]))
    print()

Neighborhood
----------------------------
125

Zoo
----------------------------
6

ATM
----------------------------
5

Airport
----------------------------
1

Airport Service
----------------------------
1

American Restaurant
----------------------------
14

Amphitheater
----------------------------
2

Antique Shop
----------------------------
7

Aquarium
----------------------------
8

Arcade
----------------------------
19

Art Gallery
----------------------------
4

Arts & Crafts Store
----------------------------
14

Arts & Entertainment
----------------------------
2

Asian Restaurant
----------------------------
43

Athletics & Sports
----------------------------
10

Australian Restaurant
----------------------------
1

BBQ Joint
----------------------------
22

Bagel Shop
----------------------------
2

Bakery
----------------------------
60

Bank
----------------------------
6

Bar
----------------------------
64

Basketball Court
----------------------------
1

Beach
--------

2

Train Station
----------------------------
45

Travel Lounge
----------------------------
1

Tree
----------------------------
1

Vegetarian / Vegan Restaurant
----------------------------
43

Video Store
----------------------------
2

Whisky Bar
----------------------------
1

Wine Bar
----------------------------
3

Wine Shop
----------------------------
13

Women's Store
----------------------------
10

Yoga Studio
----------------------------
6



In [343]:
len(df_oh[df_oh['Coffee Shop'] > 0])

81

In [344]:
len(df_oh[df_oh['Bar'] > 0])

64

In [345]:
len(df_oh[df_oh['Golf Course'] > 0])

3

In [346]:
len(df_oh[df_oh['Stadium'] > 0])

11

In [347]:
len(df_oh[df_oh['Shopping Mall'] > 0])

19

In [348]:
len(df_oh[df_oh['Supermarket'] > 0])

4

In [349]:
len(df_oh[df_oh['Department Store'] > 0])

29

In [350]:
len(df_oh[df_oh['Park'] > 0])

25

In [351]:
len(df_oh[df_oh['Fast Food Restaurant'] > 0])

77

In [352]:
len(df_oh[df_oh['Indian Restaurant'] > 0])

100

In [353]:
len(df_oh[df_oh['Asian Restaurant'] > 0])

43

In [354]:
len(df_oh[df_oh['Chinese Restaurant'] > 0])

72

In [355]:
len(df_oh[df_oh['Gym'] > 0])

42

In [356]:
len(df_oh[df_oh['Movie Theater'] > 0])

29

In [357]:
len(df_oh[df_oh['Monument / Landmark'] > 0])

9

In [358]:
len(df_oh[df_oh['Bus Station'] > 0])

15

In [359]:
len(df_oh[df_oh['Airport'] > 0])

1

In [360]:
len(df_oh[df_oh['Train Station'] > 0])

45

## Creating a new dataset with criteria for clustering

In [361]:
dffs = df_oh[['Neighborhood','Airport','Scenic Lookout','Shopping Mall','Supermarket','Convenience Store','Department Store','Electronics Store','Pharmacy','Train Station','Bus Station','Chinese Restaurant','Fast Food Restaurant','Pizza Place','Indian Restaurant','Asian Restaurant','Café','Coffee Shop','Bar','Wine Shop','Bank','ATM','College Academic Building','Beach','Monument / Landmark','Golf Course','Park','Multiplex','Movie Theater','Stadium','Club House','Gym']]
print(dffs.shape)
dffs.head()

(125, 32)


,Neighborhood,Airport,Scenic Lookout,Shopping Mall,Supermarket,Convenience Store,Department Store,Electronics Store,Pharmacy,Train Station,Bus Station,Chinese Restaurant,Fast Food Restaurant,Pizza Place,Indian Restaurant,Asian Restaurant,Café,Coffee Shop,Bar,Wine Shop,Bank,ATM,College Academic Building,Beach,Monument / Landmark,Golf Course,Park,Multiplex,Movie Theater,Stadium,Club House,Gym
0,Aarey Milk Colony,0.0,0.0,0.066667,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.066667,0.066667,0.133333,0.000000,0.066667,0.066667,0.066667,0.0,0.000000,0.0,0.0,0.0,0.066667,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.066667
1,Agripada,0.0,0.0,0.000000,0.0,0.0,0.035714,0.0,0.000000,0.035714,0.0,0.000000,0.142857,0.035714,0.214286,0.035714,0.000000,0.071429,0.035714,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.035714,0.000000,0.035714,0.071429
2,Altamount Road,0.0,0.0,0.000000,0.0,0.0,0.027778,0.0,0.000000,0.000000,0.0,0.013889,0.041667,0.041667,0.083333,0.000000,0.069444,0.055556,0.013889,0.0,0.013889,0.0,0.0,0.0,0.000000,0.0,0.027778,0.0,0.000000,0.013889,0.000000,0.000000
3,"Amboli, Mumbai",0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.038462,0.000000,0.0,0.038462,0.000000,0.076923,0.153846,0.038462,0.000000,0.076923,0.076923,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,Antop Hill,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.200000,0.300000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [362]:
dffs.sort_values(['Airport','Scenic Lookout','Shopping Mall','Supermarket','Convenience Store','Department Store','Electronics Store','Pharmacy','Train Station','Bus Station','Chinese Restaurant','Fast Food Restaurant','Pizza Place','Indian Restaurant','Asian Restaurant','Café','Coffee Shop','Bar','Wine Shop','Bank','ATM','College Academic Building','Beach','Monument / Landmark','Golf Course','Park','Multiplex','Movie Theater','Stadium','Club House','Gym'], ascending = (False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False), inplace=True)
dffs.reset_index(drop=True,inplace=True)
pd.set_option('max_rows',134)
dffs

<ipython-input-362-aeec443588d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dffs.sort_values(['Airport','Scenic Lookout','Shopping Mall','Supermarket','Convenience Store','Department Store','Electronics Store','Pharmacy','Train Station','Bus Station','Chinese Restaurant','Fast Food Restaurant','Pizza Place','Indian Restaurant','Asian Restaurant','Café','Coffee Shop','Bar','Wine Shop','Bank','ATM','College Academic Building','Beach','Monument / Landmark','Golf Course','Park','Multiplex','Movie Theater','Stadium','Club House','Gym'], ascending = (False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False), inplace=True)


,Neighborhood,Airport,Scenic Lookout,Shopping Mall,Supermarket,Convenience Store,Department Store,Electronics Store,Pharmacy,Train Station,Bus Station,Chinese Restaurant,Fast Food Restaurant,Pizza Place,Indian Restaurant,Asian Restaurant,Café,Coffee Shop,Bar,Wine Shop,Bank,ATM,College Academic Building,Beach,Monument / Landmark,Golf Course,Park,Multiplex,Movie Theater,Stadium,Club House,Gym
0,Khar Danda,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333
1,SEEPZ,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.030303,0.090909,0.090909,0.030303,0.060606,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303
2,Powai,0.000000,0.024390,0.000000,0.000000,0.000000,0.024390,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000,0.000000,0.146341,0.000000,0.024390,0.024390,0.048780,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Marine Lines,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.021739,0.000000,0.021739,0.130435,0.000000,0.065217,0.021739,0.021739,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.021739,0.021739,0.000000,0.000000,0.000000
4,Nariman Point,0.000000,0.020408,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.061224,0.020408,0.061224,0.020408,0.040816,0.040816,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.020408,0.000000,0.000000
5,Kalbadevi,0.000000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.040000,0.020000,0.060000,0.020000,0.040000,0.040000,0.020000,0.000000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000
6,Churchgate,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.030000,0.050000,0.010000,0.120000,0.020000,0.060000,0.060000,0.040000,0.000000,0.000000,0.000000,0.010000,0.010000,0.010000,0.000000,0.010000,0.020000,0.020000,0.000000,0.000000,0.000000
7,Chira Bazaar,0.000000,0.019608,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.019608,0.000000,0.019608,0.039216,0.019608,0.196078,0.000000,0.039216,0.019608,0.039216,0.019608,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.019608,0.000000,0.000000,0.019608
8,"Marine Drive, Mumbai",0.000000,0.019231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.019231,0.038462,0.019231,0.153846,0.000000,0.038462,0.019231,0.019231,0.019231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019231,0.019231,0.000000,0.000000,0.019231
9,"Mahalaxmi, Mumbai",0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.017857,0.000000,0.035714,0.035714,0.017857,0.196429,0.000000,0.053571,0.017857,0.035714,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.017857,0.000000,0.000000,0.017857


According to Strutt and Parker, a real estate consultancy group, a survey suggests that access to amenities has become on of the most important factors for house buyers.<br>
Thus, the houses are in accordance with the findings, Airport and Scenic outlook make a neighborhood prone to development. They have been given priority.<br>
For other factors, the order of precedence is as follows:<br>
1) Shopping Stores<br>
2) Transport Hub<br>
3) Restaurants<br>
4) Green Spaces<br>
5) Sports and Recreation<br>

## Calculating weighted score for Clustering

In [363]:
totc = []
for i in dffs.index:
    col_val = dffs['Airport'].loc[i]*8 + dffs['Scenic Lookout'].loc[i]*8 + dffs['Shopping Mall'].loc[i]*7 + dffs['Supermarket'].loc[i]*7 + dffs['Convenience Store'].loc[i]*7 + dffs['Department Store'].loc[i]*7 + dffs['Electronics Store'].loc[i]*7 + dffs['Electronics Store'].loc[i]*6 + dffs['Train Station'].loc[i]*6 + dffs['Bus Station'].loc[i]*6 + dffs['Chinese Restaurant'].loc[i]*5 + dffs['Fast Food Restaurant'].loc[i]*5 + dffs['Pizza Place'].loc[i]*5 + dffs['Indian Restaurant'].loc[i]*5 + dffs['Asian Restaurant'].loc[i]*5 + dffs['Café'].loc[i]*4 + dffs['Coffee Shop'].loc[i]*4 + dffs['Bar'].loc[i]*4 + dffs['Wine Shop'].loc[i]*4 + dffs['Bank'].loc[i]*3 + dffs['ATM'].loc[i]*3 + dffs['College Academic Building'].loc[i]*3 + dffs['Beach'].loc[i]*2 + dffs['Monument / Landmark'].loc[i]*1 + dffs['Golf Course'].loc[i]*1 + dffs['Park'].loc[i]*1 + dffs['Multiplex'].loc[i]*1 + dffs['Movie Theater'].loc[i]*1 + dffs['Stadium'].loc[i]*1 + dffs['Club House'].loc[i]*1 + dffs['Gym'].loc[i]*1
    totc.append(col_val)
totc[0:5]    

[1.1666666666666667,
 1.9696969696969697,
 1.6097560975609757,
 1.847826086956522,
 1.5714285714285714]

In [364]:
# assigning weights to the column
dffs['Lifestyle Score'] = totc
x = dffs[['Lifestyle Score']]
x.head()

<ipython-input-364-8f5e851008bf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dffs['Lifestyle Score'] = totc


,Lifestyle Score
0,1.166667
1,1.969697
2,1.609756
3,1.847826
4,1.571429


## K-Means Clustering

In [365]:
km = KMeans(n_clusters = 3, init = 'k-means++', random_state=5)
km.fit(x)
km.labels_

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0,
       0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 2, 2,
       0, 0, 0, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0,
       0, 1, 0, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [366]:
dffs['Cluster'] = km.labels_
y = dffs[['Cluster']]
y.head()

<ipython-input-366-e67e1c907088>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dffs['Cluster'] = km.labels_


,Cluster
0,0
1,0
2,0
3,0
4,0


## Calculating accuracy scores 

In [367]:
# training and testing datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
y = dffs['Cluster']
x_test,x_train,y_test,y_train = train_test_split(x,y,test_size=0.25,random_state=4)

In [368]:
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
km.fit(x_train)
y_h = km.predict(x_test)
print('Accuracy Score:', f1_score(y_test,y_h,average='weighted'))
print('Mean Squared Error:', mean_squared_error(y_test,y_h))
print('R-squared Value:', r2_score(y_test,y_h))

Accuracy Score: 0.9783128397729416
Mean Squared Error: 0.053763440860215055
R-squared Value: 0.9344516492810826


## Examining Clusters

In [369]:
# seeing clusters
dff = dffs[['Neighborhood','Lifestyle Score','Cluster']].sort_values('Lifestyle Score', ascending=False).reset_index()
dff.head()

,index,Neighborhood,Lifestyle Score,Cluster
0,81,Anushakti Nagar,5.000000,2
1,82,Pali Village,3.500000,2
2,52,Agripada,3.178571,2
3,78,Dava Bazaar,3.100000,2
4,49,Dedh galli,3.028571,2


In [370]:
dff.drop('index',axis=1,inplace=True)

In [371]:
# Cluster 0
dff[dff['Cluster']==0]

,Neighborhood,Lifestyle Score,Cluster
38,Lalbaug,2.368421,0
39,"Mandvi, Mumbai",2.357143,0
40,Thakurdwar,2.346939,0
41,Jagruti Nagar,2.333333,0
42,Shimpoli,2.324324,0
43,Khotachiwadi,2.315789,0
44,Bangur Nagar,2.307692,0
45,"Chor Bazaar, Mumbai",2.305556,0
46,Cavel,2.305085,0
47,Kajuwadi,2.270270,0


We can see here that Cluster 0 groups regions with an okay LifeStyle Score in the range: 1.17 - 2.37 ( approx ) 

In [372]:
# Cluster 1
dff[dff['Cluster']==1]

,Neighborhood,Lifestyle Score,Cluster
114,Lokhandwala Complex,1.000000,1
115,"Shastri Nagar, Goregaon",0.866667,1
116,Manori,0.800000,1
117,Madh Island,0.500000,1
118,Malabar Hill,0.000000,1
119,Dadar,0.000000,1
120,Fanas Wadi,0.000000,1
121,Gorai,0.000000,1
122,Kannamwar Nagar,0.000000,1
123,Koliwada,0.000000,1


We can see here that Cluster 1 groups regions with a poor Lifestyle Score. Range : 0 - 1

In [373]:
# Cluster 2
dff[dff['Cluster']==2]

,Neighborhood,Lifestyle Score,Cluster
0,Anushakti Nagar,5.000000,2
1,Pali Village,3.500000,2
2,Agripada,3.178571,2
3,Dava Bazaar,3.100000,2
4,Dedh galli,3.028571,2
5,Thakkar Bappa Colony,3.000000,2
6,Sewri,3.000000,2
7,Princess Street (Mumbai),2.915254,2
8,Lohar Chawl,2.905405,2
9,Virar,2.875000,2


We can see here that Cluster 1 groups regions with a good Lifestyle Score. Range : 2.38 - 5

## Formatting Tables for Clustering

In [374]:
# Joining tables to see latitude and longitude in neighborhoods
dff = dff.join(df_nb.set_index('Neighborhood'), on = 'Neighborhood')

In [375]:
print(dff.shape)
dff.sort_values('Cluster',inplace=True)
dff.reset_index(inplace=True)
dff.head()

(125, 5)


,index,Neighborhood,Lifestyle Score,Cluster,Latitude,Longitude
0,62,Cotton Green,2.090909,0,18.98538,72.84121
1,60,"Amboli, Mumbai",2.153846,0,19.12906,72.84644
2,59,Nala Sopara,2.166667,0,19.42000,72.81405
3,58,Churchgate,2.180000,0,18.93623,72.82682
4,57,Navy Nagar,2.181818,0,18.90603,72.81545


In [376]:
dff.drop('index',axis=1,inplace=True)
dff.head()

,Neighborhood,Lifestyle Score,Cluster,Latitude,Longitude
0,Cotton Green,2.090909,0,18.98538,72.84121
1,"Amboli, Mumbai",2.153846,0,19.12906,72.84644
2,Nala Sopara,2.166667,0,19.42000,72.81405
3,Churchgate,2.180000,0,18.93623,72.82682
4,Navy Nagar,2.181818,0,18.90603,72.81545


## Visualizing Clusters

In [382]:
import folium
lat = 19.7515
lng = 75.7139
map_final = folium.Map(location = [lat,lng], zoom_start=7)

# colors
colors = ['mediumorchid','navy','crimson']

for lat,lng,nb,clust in zip(dff['Latitude'],dff['Longitude'],dff['Neighborhood'],dff['Cluster']):
    lab = folium.Popup(str(nb) + 'Cluster:' + str(clust), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=lab,
        color=colors[clust],
        fill=True,
        fill_color=colors[clust],
        fill_opacity=0.5).add_to(map_final)
map_final

The stretch between Kalyan and Mumbai shows great growth prospect for Real Estate Investment. These conclusions are based on availability of local amenities in the mentioned neighborhoods ( within a range of 1km ).